In [10]:
import numpy as np
from sklearn import datasets
import pandas as pd
import math
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier

iris=datasets.load_iris();
x=iris.data
y=iris.target
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,y,test_size=0.5,random_state=0)

features=np.arange(1,5,1)

In [11]:
def entropy_cal(class_values,dictionary,total_length):
    ans=0
    for current_class in class_values:
        a=dictionary[current_class]/total_length
        ans=ans+a*(np.log2(a))
    if(ans==0):
        return 0
    return(-1*ans)

In [12]:
def info_gain_cal(x,y,feature,current_entropy):
    my_column=x[:,feature-1]
    np.sort(my_column)
    my_midpoint_array=[]
    for i in range(len(my_column)-1):
        a=(my_column[i]+my_column[i+1])/2
        my_midpoint_array.append(a)
        
    ans=0
    #split_info=0
    total=len(y)
    
    temp_info_gain=0
    max_info_gain=0
    final_divider=0
    
    for divider in my_midpoint_array:
        my_true_false_row1=x[:,feature-1]>divider
        my_true_false_row2=x[:,feature-1]<=divider
        
        my_y_greater_than=y[my_true_false_row1]
        my_y_smaller_than=y[my_true_false_row2]
        
        #calculating entropy of first node_greater than node
        class_values=set(my_y_greater_than)
        dictionary1={}
        for current_class in class_values:
            current_class_rows=(my_y_greater_than==current_class)
            y_current_class=my_y_greater_than[current_class_rows]
            dictionary1[current_class]=len(y_current_class)
            
        current_entropy1=entropy_cal(class_values,dictionary1,len(my_y_greater_than))
        
        c=len(my_y_greater_than)/total
        #split_info=split_info+(-1*(c)*np.log(c))
        ans=ans+c*current_entropy1
        
        #calculating entropy of first node_smaller than node
        
        class_values=set(my_y_smaller_than)
        dictionary2={}
        for current_class in class_values:
            current_class_rows=(my_y_smaller_than==current_class)
            y_current_class=my_y_smaller_than[current_class_rows]
            dictionary2[current_class]=len(y_current_class)
            
        current_entropy1=entropy_cal(class_values,dictionary2,len(my_y_smaller_than))
        
        c=len(my_y_smaller_than)/total
        #split_info=split_info+(-1*(c)*np.log(c))
        ans=ans+c*current_entropy1
        
        temp_info_gain=(current_entropy-ans)
        
        if(temp_info_gain >max_info_gain):
            max_info_gain=temp_info_gain
            final_divider=divider

    return max_info_gain,final_divider 

In [13]:
class Node:
    def __init__ (self,level,dictionary,entropy,divider,divider_feature,is_leaf):
        self.level=level
        self.dictionary=dictionary
        self.entropy=entropy
        self.divider=divider
        self.is_leaf=is_leaf
        self.divider_feature=divider_feature
        self.left=None
        self.right=None
    def print(self):
        if self.left:
            self.left.print()
        print("hi i am level",self.level)
        print()
        if self.right:
            self.right.print()

In [14]:
def decision_tree_training(x,y,features,level):
    class_values=set(y)
    dictionary={}
    #print()
    #print("level ",level)
    
    for current_class in class_values:
        current_class_rows=(y==current_class)
        y_current_class=y[current_class_rows]
        dictionary[current_class]=len(y_current_class)
        #print("count of",current_class,"=",dictionary[current_class])
    
    current_entropy=entropy_cal(class_values,dictionary,len(y))
    #print("Current Entropy is =",current_entropy)
    
    
    node=Node(level,dictionary,current_entropy,math.inf,math.inf,False)
    
    if(len(class_values)==1):
        #print("Reached leaf Node")
        node.is_leaf=True
        return node
    
    elif(len(features)==0):
        #print("Reached leaf Node")
        node.is_leaf=True
        return node
    
    max_info_gain=-1
    final_feature=0
    divider=0
    for i in features:
        temp_info_gain,temp_divider=info_gain_cal(x,y,i,current_entropy)
        if(temp_info_gain > max_info_gain):
            max_info_gain=temp_info_gain
            final_feature=i
            divider=temp_divider
    
    #print("Splitting on feature",final_feature,"with informationgain ",max_info_gain)
    node.divider=divider
    node.divider_feature=final_feature-1
    
    i=np.where(features==final_feature)
    if(len(i[0])):
        i=int(i[0])
        features=np.delete(features,i) 
        
    my_true_false_row1=(x[:,final_feature-1]>divider)
    my_true_false_row2=(x[:,final_feature-1]<=divider)
        
    my_y_greater_than=y[my_true_false_row1]
    my_y_smaller_than=y[my_true_false_row2]
    
    my_x_greater_than=x[my_true_false_row1]
    my_x_smaller_than=x[my_true_false_row2]
    
    node.left=decision_tree_training(my_x_greater_than,my_y_greater_than,features,level+1)
    node.right=decision_tree_training(my_x_smaller_than,my_y_smaller_than,features,level+1)
    
    return node
    


In [15]:
def predict_single_point(root,x):
    
    if(root.is_leaf==True):
        dictionary=root.dictionary
        classes=dictionary.keys()
        ans=0
        my_count=0
        for class_ in classes:
            if(dictionary[class_]>my_count):
                my_count=dictionary[class_]
                ans=class_
        return ans
    
    feature_index=root.divider_feature
    feature_value=x[feature_index]
    ans=0
    
    if(feature_value >root.divider):
        ans=predict_single_point(root.left,x)
    else:
        ans=predict_single_point(root.right,x)
    
    return ans

In [16]:
def predict(root,x_test):
    y_pred=[]
    for x in x_test:
        x_class=predict_single_point(root,x)
        y_pred.append(x_class)
    return y_pred

In [17]:
root=decision_tree_training(x_train,y_train,features,0)
y_prediction=predict(root,x_test)

print(y_prediction)
print()


[2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 2, 0, 0, 2, 1, 0, 0, 1]



In [18]:
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(y_test,y_prediction))
print(confusion_matrix(y_test,y_prediction))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        21
          1       0.91      1.00      0.95        30
          2       1.00      0.88      0.93        24

avg / total       0.96      0.96      0.96        75

[[21  0  0]
 [ 0 30  0]
 [ 0  3 21]]
